# Comparing methods of hurricane forecast uncertainty
##### author: Elizabeth A. Barnes, Randal J. Barnes and Mark DeMaria
##### version: v0.1.0

```
conda create --name env-hurr-tfp python=3.9
conda activate env-hurr-tfp
pip install tensorflow==2.7.0
pip install tensorflow-probability==0.15.0
pip install --upgrade numpy scipy pandas statsmodels matplotlib seaborn 
pip install --upgrade palettable progressbar2 tabulate icecream flake8
pip install --upgrade keras-tuner sklearn
pip install --upgrade jupyterlab black isort jupyterlab_code_formatter
pip install silence-tensorflow
pip install tqdm
```

Use the command
```python -m pip freeze > requirements.txt```
to make a pip installation list.

In [ ]:
import datetime
import os
import pickle
import pprint
import time

import experiment_settings
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from build_data import build_hurricane_data
import build_model
from model_diagnostics import plot_history
from save_model_run import save_model_run
from sklearn import preprocessing
from training_instrumentation import TrainingInstrumentation
from silence_tensorflow import silence_tensorflow
import tensorflow_probability as tfp

silence_tensorflow()

In [ ]:
__author__ = "Randal J Barnes and Elizabeth A. Barnes"
__version__ = "21 January 2022"

EXP_NAME_LIST = (
                 "intensity400_EPCP72",
                )


OVERWRITE_MODEL = True
DATA_PATH = "data/"
MODEL_PATH = "saved_models/"

In [ ]:
mpl.rcParams["figure.facecolor"] = "white"
mpl.rcParams["figure.dpi"] = 150
np.warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

## Start looping through random seeds

In [ ]:
for EXP_NAME in EXP_NAME_LIST:
    settings = experiment_settings.get_settings(EXP_NAME)
    # pprint.pprint(settings, width=80)

    RNG_SEED_LIST = np.copy(settings['rng_seed'])
    # NETWORK_SEED_LIST = [1701, 5552, 8529, 4440, 2657,] # currently over-written later in this script

    for rng_seed in RNG_SEED_LIST:
        settings['rng_seed'] = rng_seed
        # pprint.pprint(settings['rng_seed'], width=80)

        # build the intensity data tensors
        (
            data_summary,
            x_train,
            onehot_train,
            x_val,
            onehot_val,
            x_test,
            onehot_test,    
            df_train,
            df_val,
            df_test,
        ) = build_hurricane_data(DATA_PATH, settings, verbose=2)

        # define the callbacks
        earlystoping_callback = tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            mode="min",
            patience=settings["patience"],
            restore_best_weights=True,
            verbose=1,
        )

        training_callback = TrainingInstrumentation(
            x_train,
            onehot_train,
            interval=50,
        )

        callbacks = [earlystoping_callback, 
                     training_callback,
                    ]

        # set network seed and train the model
        NETWORK_SEED_LIST = [settings["rng_seed"]]

        for network_seed in NETWORK_SEED_LIST:
            tf.random.set_seed(network_seed)  # This sets the global random seed.

            # Create the model name.
            model_name = (
                EXP_NAME + "_" + settings["uncertainty_type"] + '_' + f"network_seed_{network_seed}_rng_seed_{settings['rng_seed']}"
            )
            pprint.pprint(model_name)

            # Make, compile, and train the model
            tf.keras.backend.clear_session()            
            model = build_model.make_model(
                settings,
                x_train,
                onehot_train,
                model_compile=True,
            )   
            model.summary()

            # check if the model exists
            model_savename = MODEL_PATH + model_name + "_weights.h5"
            if os.path.exists(model_savename) and OVERWRITE_MODEL==False:
                print(model_savename + 'exists. Skipping...')
                continue

            # train the network
            start_time = time.time()
            history = model.fit(
                x_train,
                onehot_train,
                validation_data=(x_val, onehot_val),
                batch_size=settings["batch_size"],
                epochs=settings["n_epochs"],
                shuffle=True,
                verbose=0,
                callbacks=callbacks,
            )
            stop_time = time.time()

            # Display the results, and save the model rum.
            best_epoch = np.argmin(history.history["val_loss"])
            fit_summary = {
                "network_seed": network_seed,
                "elapsed_time": stop_time - start_time,
                "best_epoch": best_epoch,
                "loss_train": history.history["loss"][best_epoch],
                "loss_valid": history.history["val_loss"][best_epoch],
            }
            pprint.pprint(fit_summary, width=80)
            plot_history(history, model_name)

            save_model_run(
                data_summary,
                fit_summary,
                model,
                MODEL_PATH,
                model_name,
                settings,
                __version__,
            )